# PID Controller Simulation

* Simulation provides functional evidence and proof of the code
* Simulation allows inspection of graphical curse of PID controller
* Use tge very **same code** from steer_and_control crate **for simulation** as on **embedded target**


In [ ]:
:dep plotters = { git = "https://github.com/38/plotters", default_features = false, features = ["evcxr", "all_series", "all_elements"] }
:dep conreg = { verion = "0.1.0", default_features = false, features = ["std"] }
:sccache = 1

In [ ]:
use conreg::control::{
    pid::PidController,
    pid_parameter::PidParameterAdditive
};
use std::iter;
use plotters::prelude::*;
use plotters::prelude::LineSeries;

In [ ]:
// instanciate the PID controller
let p = PidParameterAdditive::new(1.0).set_integral(2.0).set_differential(0.5);
let mut pid = PidController::<f32>::new_with_t1(0.01, 0.1).set(p);
println!("PID controller {:?}", pid );

In [ ]:
// compose step response:
let step_pre = iter::repeat(0.0_f32).take(100);
let step_post = iter::repeat(1.0_f32).take(1101);
let step_y = step_pre.chain(step_post);

In [ ]:
// compute the simulation data
let setpoint = step_y.clone();
let process = iter::repeat(0.0_f32).take(1201);
let manipulated = setpoint.zip(process)
    .map(|x| pid.control(x)) // run the controller
    .collect::<Vec<f32>>();  // this collector needs to be called to compensate for map function

In [ ]:
// configure the simulation plot
let figure = evcxr_figure((640, 480), |root| {
    root.fill(&WHITE);
    let mut chart = ChartBuilder::on(&root)
        .caption("Step Response of PID(1.0, 2.0s, 0.5/s)-T1(0.1s)", ("Arial", 24).into_font())
        .margin(5)
        .x_label_area_size(30)
        .y_label_area_size(30)
        .build_cartesian_2d(-0.1f32..1.05f32, -0.1f32..6.0f32)?;

    chart.configure_mesh().draw()?;

    chart.draw_series(LineSeries::new(
        (-100..=1100).map(|x| x as f32 * 0.01).zip(step_y),
        &RED,
    )).unwrap()
        .label("Step Function")
        .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &RED));
    chart.draw_series(LineSeries::new(
        (-100..=1099).map(|x| x as f32 * 0.01).zip(manipulated),
        &BLUE,
    )).unwrap()
        .label("PID Response on step func")
        .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &BLUE));

    chart.configure_series_labels()
        .background_style(&WHITE.mix(0.8))
        .border_style(&BLACK)
        .position(SeriesLabelPosition::UpperRight)
        .draw()?;
    Ok(())
});

In [ ]:
// show the plot
figure